Βήμα 1: Συλλογή Δεδομένων

Ο παρακάτω κώδικας με την χρήση της Beautiful soup λαμβάνει τον κώδικα html από την σελίδα της wikipedia. Με την entry.find() βρίσκει μεσα στον Html κώδικα τον τίτλο, ημερομηνία, πληροφορίες, Links και αλλές παρόμοιες πληροφορίες. Για την επεξεργασία των αποτελεσμάτων δημιουργείται μια κενή λίστα, η data και η temp_id μεταβλητή, η οποία κάθε φορά αυξάνεται και αντιπροσωπεύει το ID των εγγράφων. Δημιουργείται ένας βρόχος, στον οποίο διατρέχονται τα αποτελέσματα της αναζήτησης. Εντοπίζεται ο τίτλος και η ημερομηνία του εγγράφου. Αν υπάρχει ο τίτλος και η ημερομηνία, τότε διαχωρίζεται ο τίτλος του εγγράφου, διαχωρίζει το date_element σε τμήματα με βάση τον χαρακτήρα '-' ή την αλλαγή γραμμής (\n) και αποθηκεύονται οι πληροφορίες του εγγράφου (σχετίζονται με το μέγεθος του εγγράφου) και η ημερομηνία του εγγράφου (πότε εκδόθηκε). Τα δεδομένα του info και του date είναι το πρώτο στοιχείο και το δεύτερο στοιχείο αντίστοιχα του date_parts χωρίς του λευκούς χαρακτήρες. Στην συνέχεια, εντοπίζεται το <a>, έτσι ώστε να συλλέξουμε τον σύνδεσμο και να δημιουγηθεί το πλήρες URL του εκάστοτε άρθρου. Μετά κατευθύνεται στο εκάστοτε άρθρο μέσω του Link απο το href και συλλέγει το περιεχόμενο του με την επανάληψη του beautiful soup στην συγκεκριμένη σελίδα. Για κάθε άρθρο εκχωρείται ένα μοναδικό ID, το οποίο μόλις εκχωρηθεί η τιμή, αυξάνεται η μεταβλητή temp_id κατά ένα. Τέλος, όλα τα δεδομένα εισάγονται στην λίστα data με την εντολή append και αυτά τέλος αποθηκεύονται σε αρχείο με όνομα results.json.

In [5]:
import requests
from bs4 import BeautifulSoup
import json
import time
import re

url ='https://en.wikipedia.org/w/index.php?search=heroes+of+1821+greece&title=Special%3ASearch&ns0=1'
html = requests.get(url)
soup = BeautifulSoup(html.content, 'html.parser')
entries =soup.find_all('li', class_='mw-search-result mw-search-result-ns-0')
#print(entries)
data = []
temp_id = 1

for entry in entries: 
    title_element = entry.find('div', class_='mw-search-result-heading')
    date_element = entry.find('div', class_='mw-search-result-data')

    if title_element and date_element:
        first_title = title_element.text.strip()
        date_parts = re.split(r'[-\n]', date_element.text.strip())
        infos = date_parts[0].strip() if len(date_parts) > 1 else "N/A"
        date = date_parts[1].strip() if len(date_parts) > 1 else "N/A"
        
       
        link_element = title_element.find('a')
        
        if link_element and 'href' in link_element.attrs:
            content_url = 'https://en.wikipedia.org' + link_element['href']
            ID = temp_id
            temp_id = temp_id + 1
        else:
            print("No href attribute found")
            continue  

        content_html = requests.get(content_url)
        content_soup = BeautifulSoup(content_html.content, 'html.parser')
        content_element = content_soup.find('div', class_='mw-content-ltr mw-parser-output')
        
        content = content_element.text.strip() if content_element else "N/A"

        data.append([first_title, ID, infos, date, content])
        print(f'First_Title: {first_title}\n\n ID: {ID}n\\n Informations: {infos}\n\n Date: {date}\n\nContent: {content}\n{"-" * 50}')
    
    time.sleep(1)


json_filename = "results.json"
with open(json_filename, 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=2)

print(f"Τα δεδομένα αποθηκεύτηκαν στο αρχείο: {json_filename}")


First_Title: Greek War of Independence

 ID: 1n\n Informations: 175 KB (21,181 words)

 Date: 13:44, 18 January 2025

Content: Greek rebellion against the Ottoman Empire (1821–1829)


Greek War of IndependencePart of the Revolutions during the 1820sTop left: The camp  at Phaliro. Top right: The burning of an Ottoman frigate by a Greek fire ship. Bottom right: The Battle of Navarino. Bottom left: Ibrahim Pasha of Egypt at the Third Siege of Missolonghi.Date21 February 1821 – 12 September 1829[2](8 years, 6 months and 3 weeks)LocationOttoman Greece (present-day Greece)Result
Greek victory[a]Territorialchanges
Independence of Greece

The Peloponnese, Saronic Islands, Cyclades, Sporades and Continental Greece ceded to the independent Greek state
Crete ceded to EgyptBelligerents
 Greek Revolutionaries


In detail:
 Klephts
 Armatoloi
 Filiki Eteria [nb 1]
 Sacred Band [nb 1]
 Messenian Senate [nb 1]
 Peloponnesian Senate [nb 2]
 Senate of Western Continental Greece [nb 2]
 Areopagus of East

Βήμα 2. Προεπεξεργασία κειμένου (Text Processing)

Με την χρήση της βιβλιοθήκης Pandas και τη συνάρτηση της βιβλιοθήκης read_json διαβάζεται το έγγραφο results.json, το οποίο περιλαμβάνει μέσα τα 
περιεχόμενα, τίτλο και όλες τις απαραίτητες πληροφορίες από το προηγούμενο ερώτημα και καταχωρείται στην μεταβλητή df. Για κάθε στήλη που περιλαμβάνεται στην μεταβλητή εκτελείται η συνάρτηση που έχει δημιουργηθεί με όνομα Preprocess_text. Η συνάρτηση preprocess_text λαμβάνει την παράμετρο text την μετατρέπει σε token με την χρήση της βιβλιοθήκης nltk.tokenize και φτιάχνει μία άδεια λίστα processed_words. Για κάθε λέξη μέσα στην λίστα words ελέγχει αν είναι αριθμός. Αν είναι την μετατρέπει σε string και την βάζει στην λίστα της processed_words, χωρίς περαιτέρω επεξεργασία. Αλλιώς, εκτελεί τις παρακάτω διαδικασίες: Αφαιρεί απο την λέξη τα σημεία στίξης. Γίνεται το stemming, χρησιμοποιώντας τον αλγόριθμο PorterStemmer(). Στο  stemming κόβονται οι καταλήξεις απο τις λέξεις. Έπειτα, υλοποιείται το lemmatization, χρησιμοποιώντας τον αλγόριθμο WordNetLemmatizer(). Το lemmatization απλοποιεί τις λέξεις. Ακόμα, σβήνονται τα stop words και τα προσθέτει ως πεζά στο processed_words και επιστρέφεται η Processed_words με κενά ανάμεσα στις λέξεις στο main πρόγραμμα. Στην συνέχεια ορίζονται οι νέες στήλες και μετονομάζεται το df με τις καινούργιες στήλες από το new_columns. Τέλος, ελέγχει αν υπάρχει η στήλη text και εφαρμόζεται regex στη στήλη Content, έτσι ώστε να αφαιρεθούν οι μη ascii χαρακτήρες (εφόσον υπάρχει). Όλα αποθηκεύονται σε ένα νέο Json αρχείο με όνομα processed.

In [7]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string

def preprocess_text(text):
    #tokenization
    words = word_tokenize(str(text))  # Μετατρέπουμε σε string για να αντιμετωπίσουμε ακέραιους
    
    processed_words = []
    
    for word in words:
        #Ελέγχουμε αν η λέξη είναι ακέραιος
        if word.isdigit():
            processed_words.append(str(word))  # Αποθηκεύουμε τον ακέραιο ως string
        else:
            #remove punctuation( Αφαίρεση σημείων στίξης)
            word = word.strip(string.punctuation)
            
            #stemming(κοβω τις καταληξεις)
            stemmer = PorterStemmer()
            word = stemmer.stem(word)
            
            #lemmatization(απλοποιω τις λεξεις)
            lemmatizer = WordNetLemmatizer()
            word = lemmatizer.lemmatize(word)
            
            #remove stop words(σβήνω λέξεις χωρίς νόημα)
            stop_words = set(stopwords.words('english'))
            if word.lower() not in stop_words:
                processed_words.append(word.lower())
    
    return ' '.join(processed_words)

#διαβάζω το αρχείο results.json
# Πρέπει να ορίστει το μονοπάτι στο οποίο τρέχει κάθε φορά το πρόγραμμα,
# αλλιώς εμφανίζει λάθος
df = pd.read_json(r'C:\Users\smoul\Anaktisi\results.json')


#εφαρμόζω την προεπεξεργασία σε κάθε κελί του πίνακα
for column in df.columns:
    df[column] = df[column].apply(preprocess_text)

#ορίζει τα νέα ονόματα για τις στήλες
new_columns = {
    0: 'First_Title',
    1: 'ID',
    2: 'Informations',
    3: 'Date',
    4: 'Content',
}

#μετονομάζει τις στήλες χρησιμοποιώντας το λεξικό που ορίσαμε
df = df.rename(columns=new_columns)

# Εφαρμογή του regex στη στήλη 'Content', αν υπάρχει
if 'Content' in df.columns:  # Ελέγχει αν υπάρχει στήλη 'text'
    df['Content'] = df['Content'].apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', str(x)))

# Αποθηκεύω το νέο αρχείο processed.json
json_filename = "processed.json"
with open(json_filename, 'w', encoding='utf-8') as json_file:
    # Χρησιμοποιούμε τη μέθοδο to_json της pandas
    df.to_json(json_file, force_ascii=False, indent=2, orient="records")

print("Η επεξεργασία ολοκληρώθηκε και τα δεδομένα αποθηκεύτηκαν στο αρχείο processed.json.")


Η επεξεργασία ολοκληρώθηκε και τα δεδομένα αποθηκεύτηκαν στο αρχείο processed.json.


Βήμα 3. Ευρετήριο (Indexing)

Το πρόγραμμα ξεκινάει με το φόρτωμα του αρχείου processed.json σε μια λίστα documents. Δημιουργούνται μια κένη λίστα, η οποία συλλέγει όλους τους όρους από όλα τα έγγραφα, και τριά κενά λεξικά, τα οποία είναι τα inverted_index (είναι το ανεστραμμένο ευρετηρίο, το οποίο για κάθε όρο εμφανίζει τη λίστα των εγγράφων), boolean_inverted_index (είναι το ανεστραμμένο ευρετηρίο, το οποίο για κάθε όρο εμφανίζει τη λίστα των εγγράφων, αλλά σε τιμές 0 και 1) και temp. Δημιουργείται ένας βρόχος, ο οποίος διατρέχει το κάθε έγγραφο και εκτελεί τις παρακάτω διαδικασίες: Αρχικά, ανακτάται το ID του κάθε εγγράφου. Δημιουργείται ένας βρόχος, ο οποίος διατρέχει όλα τα πεδία του εγγράφου, παραλείποντας το πεδίο ID. Ελέγχεται αν η τιμή του πεδίου είναι string. Εφόσον είναι, τότε το κείμενο διαχωρίζεται σε λέξεις (tokens) με βάση τα κενά διαστήματα και συγκεντρώνει όλους τους όρους σε μια λίστα. 
Έπειτα, δημιουργείται το αντεστραμμένο ευρετήριο. Δημιουργείται ένα βρόχος, που διατρέχει όλους τους όρους. Γίνεται έλεγχος αν ο όρος υπάρχει στο inverted_index. Αν υπάρχει, τότε προστίθεται το ID του εγγράφου στην λίστα των εγγράδφων όπου εμφανίζεται ο όρος. Αλλίως, δημιουργείται μια νέα καταχώριση στο ευρετήριο για τον όρο, με τιμή μια λίστα που περιέχει το τρέχον doc_id. Μέσα στο Inverted_index για
κάθε όρο αφαιρούνται τα διπλότυπα, ταξινομούνται αριθμητικά με την εντολή sorted, γίνεται ενημέρωση του inverted_index και μετατρέπεται η τιμή του όρου σε λίστα (είχε αποκτήσει τον τύπο set). Επιπρόσθετα, δημιουργείται το λογικό αντεστραμμένο ευρετήριο. Αρχικά, για κάθε όρο στο inverted_index, αποθηκεύεται η λίστα με τα IDs των εγγράφων που περιέχουν τον όρο στη μεταβλητή temp. Αν ο όρος, δεν υπάρχει, τότε δημιουργείται μια νέα καταχώρηση. Στη συνέχεια, εξετάζεται η παρουσία του όρου στα έγγραφα. Αν το ID είναι υπάρχει στην λίστα του αντεσταμμένου ευρετηρίου, τότε προστίθεται ο χαρακτήρας 1, αλλιώς προστίθεται ο χαρακτήρας 0. Τέλος, αποθηκεύονται οι Inverted index και boolean inverted index ως json σε αντίστοιχα αρχεία inverted_index.json και boolean_inverted_index.json.


In [16]:
import json

# Διαβάζω το αρχείο JSON
with open(r'C:\Users\smoul\Anaktisi\processed.json', 'r') as f:
    documents = json.load(f)  # Φορτώνω τα δεδομένα

# Λίστα που θα περιέχει όλους τους όρους
all_terms = []

# Αντίστροφο ευρετήριο
inverted_index = {}
boolean_inverted_index = {}
temp = {}

# Επεξεργασία κάθε εγγράφου
for document in documents:
    doc_id = document.get('ID')
    for field, field_value in document.items():
        if field == "ID":
            continue
        if isinstance(field_value, str):  # Ελέγχουμε αν το πεδίο είναι string
            terms = field_value.split()
            all_terms.extend(terms)  # Προσθήκη όρων στη λίστα για όλα τα έγγραφα

            # Δημιουργία του ανεστραμμένου ευρετηρίου
            for term in terms:
                if term in inverted_index:
                    inverted_index[term].append(doc_id)
                else:
                    inverted_index[term] = [doc_id]
    
for term in inverted_index:
    unique_ids = set(inverted_index[term])  # Αφαιρούμε τα διπλότυπα
    sorted_ids = sorted(unique_ids, key=int)  # Ταξινομούμε αριθμητικά
    inverted_index[term] = sorted_ids  # Μετατροπή σε λίστα και ενημέρωση του inverted_index
    
for term in inverted_index:
    temp = inverted_index[term]  # Ταξινομημένη λίστα με IDs
    if term not in boolean_inverted_index:
        boolean_inverted_index[term] = []
    
    # Δημιουργούμε Boolean λίστα για 20 πιθανές θέσεις
    for index in range(20):  # Έλεγχος για 20 εγγραφές
        if str(index + 1) in temp:  # Ελέγχουμε αν το index + 1 υπάρχει στο temp
            boolean_inverted_index[term].append('1')  # Προσθήκη 1
        else:
            boolean_inverted_index[term].append('0')  # Προσθήκη 0
            
# Αποθήκευση του ανεστραμμένου ευρετηρίου σε ένα αρχείο JSON
with open('inverted_index.json', 'w') as json_file:
    json.dump(inverted_index, json_file)
    
# Αποθήκευση του ανεστραμμένου ευρετηρίου σε ένα αρχείο JSON
with open('boolean_inverted_index.json', 'w') as json_file:
    json.dump(boolean_inverted_index, json_file)

# Δοκιμαστική εκτύπωση μερικών όρων
print(all_terms[:10])

['greek', 'war', 'independ', '175', 'kb', '21,181', 'word', '13:44', '18', 'januari']


Βήμα 4 (Μηχανή αναζήτησης (Search Engine) και Βήμα 5 (Αξιολόγηση συστήματος)

Ο παρακάτω κώδικας υλοποιεί το βήμα 4 και το βήμα 5. Στο βήμα 4 ο χρήστης εισάγει το ερώτημα του και εξάγει τα συναφή έγγραφα, ενώ στο βήμα 5 αξιολογείται η μηχανή αναζήτησης με τις μετρικές που έχουν αναφερθεί στην αναφορά.
    Στο βήμα 4 επεξεργάζεται το ερώτημα του χρήστη. Χωρίζεται σε 2 μέρη. Στο πρώτο μέρος επεξεργάζεται το ερώτημα του χρήστη και στην συνέχεια υλοποιεί Boolean πράξεις. 
    Αρχικά, υλοποιείται η συνάρτηση preprocess_text, η οποία πεξεργαζεται το ερώτημα του χρήστη. Η συγκεκριμένη συνάρτηση δέχεται ως όρισμα ένα κείμενο (είναι το ερώτημα του χρήστη) και χρησιμοποιώντας την συνάρτηση word_tokenize δημιουργεί μια λίστα με τις λέξεις του ορίσματος. Αυτές οι λέξεις μετατρέπονται σε string και αποθηκεύονται στην μεταβλητή words. Δημιουργείται μια λίστα η processed_words, στην οποία θα αποθηκεύονται οι επεξεργασμένες λέξεις, η οποία στο τέλος επιστρέφεται. Για να επεξεργαστούν όλες οι λέξεις, δημιουργείται ένας βρόχος, στον οποίο:
•	Γίνεται ένας έλεγχος, αν η λέξη είναι αριθμός. Αν είναι αληθής ο έλεγχος, τότε προστίθεται στην λίστα processed_words, χωρίς περαιτέρω επεξεργασία. 
•	Αν δεν ισχύει ο έλεγχος, τότε:
a)	Αφαιρούνται όλα τα σημεία στίξης από την λέξη. Χρησιμοποιείται η  string.punctuation (περιέχεται στην βιβλιοθήκη string), η οποία περιέχει όλους τους χαρακτήρες στίξης.  Η μέθοδος strip αφαιρεί όλα τα σημεία στίξης από την αρχή έως το τέλος του string.
b)	Αφαιρούνται οι καταλήξεις από τις λέξεις, χρησιμοποιώντας τον αλγόριθμο  PorterStemmer.
c)	Αναζητάται η ρίζα των λέξεων, χρησιμοποιώντας τον αλγόριθμο  WordNetLemmatizer.
d)	Αφαιρούνται οι κοινές λέξεις, οι stop words. 
    Στη συνέχεια, υλοποιείται η συνάρτηση load_inverted_index_for_boolean. Η συγκεκριμένη συνάρτηση έχει ως στόχο να φορτώσει το αντεστραμμένο ευρετήριο που περιέχει Boolean τιμές, δηλαδή τις τιμές 0 και 1. Δέχεται ως όρισμα το μονοπάτι που βρίσκεται το αρχείο. Σε περίπτωση που δεν δοθεί κάποια τιμή, τότε χρησιμοποιεί το προεπιλεγμένο όνομα αρχείου που βρίσκεται στον ίδιο φάκελο με το αρχείο boolean_inverted_index.json. Ανοίγει το αρχείο για να το διαβάσει, χρησιμοποιώντας την επιλογή ‘r’ και την συνάρτηση json.load για να διαβάσει τα δεδομένα και να τα αποθηκεύσει στην μορφή λεξικό. Αν το αρχείο βρεθεί, τότε επιστρέφεται το περιεχόμενο του. Αλλιώς εμφανίζεται το μήνυμα λάθους, το οποίο αναφέρει ότι το αρχείο δεν υπάρχει.
Ακόμα, υλοποιείται η συνάρτηση load_processed. Η συγκεκριμένη συνάρτηση έχει ως στόχο να φορτώσει το αρχείο που περιέχει τις επεξεργασμένες λέξεις. Δέχεται ως όρισμα το μονοπάτι που βρίσκεται το αρχείο. Σε περίπτωση που δεν δοθεί κάποια τιμή, τότε χρησιμοποιεί το προεπιλεγμένο όνομα αρχείου που βρίσκεται στον ίδιο φάκελο με το αρχείο processed.json. Ανοίγει το αρχείο για να το διαβάσει, χρησιμοποιώντας την επιλογή ‘r’ και την συνάρτηση json.load για να διαβάσει τα δεδομένα και να τα αποθηκεύσει στην μορφή λεξικό. Αν το αρχείο βρεθεί, τότε επιστρέφεται το περιεχόμενο του. Αλλιώς εμφανίζεται το μήνυμα λάθους, το οποίο αναφέρει ότι το αρχείο δεν υπάρχει.
        Έπειτα, υλοποιείται η συνάρτηση load_inverted_index. Η συγκεκριμένη συνάρτηση έχει ως στόχο να φορτώσει το αντεστραμμένο ευρετήριο που περιέχει τα ID των εγγράφων. Δέχεται ως όρισμα το μονοπάτι που βρίσκεται το αρχείο. Σε περίπτωση που δεν δοθεί κάποιο μονοπάτι, τότε χρησιμοποιεί το προεπιλεγμένο όνομα αρχείου που βρίσκεται στον ίδιο φάκελο με το αρχείο inverted_index.json. Ανοίγει το αρχείο για να το διαβάσει, χρησιμοποιώντας την επιλογή ‘r’ και την συνάρτηση json.load για να διαβάσει τα δεδομένα και να τα αποθηκεύσει στην μορφή λεξικό. Αν το αρχείο βρεθεί, τότε επιστρέφεται το περιεχόμενο του. Αλλιώς εμφανίζεται το μήνυμα λάθους, το οποίο αναφέρει ότι το αρχείο δεν υπάρχει. 
Επίσης, υλοποιείται η συνάρτηση preprocess_file. Η συγκεκριμένη συνάρτηση έχει ως στόχο να αφαιρέσει τα περιττά στοιχεία από το Content του εκάστοτε αρχείου. Δέχεται ωε όρισμα ένα κείμενο και στη συνέχεια, το επεξεργάζεται μετατρέποντας το σε συμβολοσειρά και χωρίζοντας τις λέξεις που περιέχει. Δημιουργείται μια λίστα η processed_words, στην οποία θα αποθηκεύονται οι επεξεργασμένες λέξεις, η οποία στο τέλος επιστρέφεται. Δημιουργείται ένας βρόχος, ο οποίος διατρέχει όλες τις λέξεις. Μέσα στον βρόχο υλοποιούνται κάποιες διαδικασίες. Αρχικά, γίνεται ένας έλεγχος, στον οποίο ελέγχεται αν η λέξη είναι αριθμός. Αν είναι, τότε αποθηκεύεται ως string στην λίστα  processed_words. Αν δεν είναι αριθμός η λέξη, τότε αφαιρούνται όλα τα σημεία στίξης από την αρχή και το τέλος της λέξης και στην συνέχεια, ελέγχεται αν είναι κενή μετά την αφαίρεση των σημείων στίξης. Εφόσον, δεν είναι κενή τότε αποθηκεύεται στην λίστα processed_words με πεζά γράμματα. Στο τέλος, επιστρέφεται η λίστα ξανά σε κείμενο, στο οποίο οι λέξεις ενώνονται με τον χαρακτήρα ‘κενό’. 
        Μετά, υλοποιείται η συνάρτηση load_results. Η συγκεκριμένη συνάρτηση έχει ως στόχο να επεξεργαστεί το κείμενο και να το εμφανίσει με μια ομοιόμορφη κατάταξη. Δέχεται ως όρισμα το μονοπάτι του αρχείου που πρόκειται να φορτωθεί. Σε περίπτωση που δεν δοθεί το μονοπάτι, τότε χρησιμοποιεί το προεπιλεγμένο όνομα αρχείου που βρίσκεται στον ίδιο φάκελο με το αρχείο results.json. Το αρχείο φορτώνεται σε ένα DataFrame της pandas. Κάθε κλειδί του JSON αντιστοιχεί σε μία στήλη και κάθε τιμή σε γραμμή. Δημιουργείται ένα βρόχος, ο οποίος επεξεργάζεται το κάθε κελί της στήλης εφαρμόζοντας την συνάρτηση preprocess_file (επεξηγείται παραπάνω). Έξω από το βρόχο οι στήλες του DataFrame μετονομάζονται και τα αριθμητικά κλειδιά αντικαθίστανται με τα ονόματα: First_Title, ID,  Informations, Date και  Content. Γίνεται έλεγχος, αν η στήλη Content υπάρχει στο DataFrame. Αν υπάρχει, τότε αφαιρούνται όλοι οι χαρακτήρες που δεν είναι ascii, όπως τα ελληνικά. Τέλος, το DataFrame επιστρέφεται ως λεξικό, το οποίο περιέχει τα ονόματα των στηλών ως κλειδιά. Τα παραπάνω βήματα υλοποιούνται εφόσον υπάρχει το αρχείο. Αν το αρχείο δεν υπάρχει, τότε εμφανίζεται μήνυμα λάθους ότι αρχείο δεν βρέθηκε και επιστρέφεται None.
        Τέλος, για το πρώτο μέρος, υλοποιούνται οι Boolean πράξεις. Ειδικότερα, υλοποιούνται οι λογικές πράξεις AND, OR και NOT. Η πράξη AND βρίσκει τα κοινά έγγραφα που υπάρχουν στους όρους του ερωτήματος. Αρχικά, γίνεται έλεγχος αν το ερώτημα περιέχει την λέξη AND. Αν την περιέχει, τότε αντικαθιστάται η λέξη με το κενό χαρακτήρα. Το ερώτημα χωρίζεται σε λέξεις, απαλείφονται οι κενοί χαρακτήρες και επεξεργάζονται οι όροι του ερωτήματος με χρήση της συνάρτησης preprocess_text. Στην συνέχεια, χρησιμοποιείται η filter για να αφαιρεθούν οι κενοί χαρακτήρες και όλοι οι όροι αποθηκεύονται στην λίστα terms. Αρχικοποιείται, η μεταβλητή result_and με 20 χαρακτήρες με την τιμή 1. Δημιουργείται ένας βρόχος, ο οποίος υλοποιεί τις παρακάτω διαδικασίες. Ελέγχει αν ο όρος υπάρχει στο inverted_index. Αν υπάρχει, τότε γίνεται η λογική πράξη AND. Χρησιμοποιούνται τα bitstrings των όρων. Τέλος, εκτυπώνεται το αποτέλεσμα του AND. Η πράξη OR βρίσκει τα έγγραφα που υπάρχουν στους όρους του ερωτήματος (κάνει την ένωση). Αρχικά, γίνεται έλεγχος αν το ερώτημα περιέχει την λέξη OR. Αν την περιέχει, τότε αντικαθιστάται η λέξη με το κενό χαρακτήρα. Το ερώτημα χωρίζεται σε λέξεις, απαλείφονται οι κενοί χαρακτήρες και επεξεργάζονται οι όροι του ερωτήματος με χρήση της συνάρτησης preprocess_text. Στην συνέχεια, χρησιμοποιείται η filter για να αφαιρεθούν οι κενοί χαρακτήρες και όλοι οι όροι αποθηκεύονται στην λίστα terms. Αρχικοποιείται, η μεταβλητή result_or με 20 χαρακτήρες με την τιμή 0. Δημιουργείται ένας βρόχος, ο οποίος υλοποιεί τις παρακάτω διαδικασίες. Ελέγχει αν ο όρος υπάρχει στο inverted_index. Αν υπάρχει, τότε αποθηκεύονται στην λίστα term_result η τιμή που αντιστοιχεί στο κλειδί με τον συγκεκριμένο όρο. Έπειτα, για κάθε θέση της λίστας επιλέγεται η μεγαλύτερη τιμή ανάμεσα στις θέσεις του result_or και στο term_result. Τέλος, εκτυπώνεται το αποτέλεσμα του OR. Τέλος, η πράξη NOT βρίσκει τα έγγραφα που υπάρχουν στον έναν όρο, αλλά δεν υπάρχουν στους άλλους του ερωτήματος. Αρχικά, γίνεται έλεγχος αν το ερώτημα περιέχει την λέξη NOT. Αν την περιέχει, τότε αντικαθιστάται η λέξη με το κενό χαρακτήρα. Το ερώτημα χωρίζεται σε λέξεις, απαλείφονται οι κενοί χαρακτήρες και επεξεργάζονται οι όροι του ερωτήματος με χρήση της συνάρτησης preprocess_text. Στην συνέχεια, χρησιμοποιείται η filter για να αφαιρεθούν οι κενοί χαρακτήρες και όλοι οι όροι αποθηκεύονται στην λίστα terms. Αρχικοποιείται, η μεταβλητή result_not με 20 χαρακτήρες με την τιμή 1. Δημιουργείται ένας βρόχος, ο οποίος υλοποιεί τις παρακάτω διαδικασίες. Ελέγχει αν ο όρος δεν υπάρχει στο inverted_index. Αν δεν υπάρχει, επιστρέφονται 2 κενές λίστες. Στην συνέχεια, το result_not ενημερώνεται κάθε φορά με τη χρήση AND που περιγράφεται στην διαδικασία του AND. Έπειτα, γίνεται αντιστροφή του result_not. Τέλος, γίνεται ένας έλεγχος, στον οποίο το αποτέλεσμα του result_not βγάλει 1 για ένα έγγραφο, στο οποίο δεν περιέχεται ο όρος και εκτυπώνεται το αποτέλεσμα του NOT. Ακόμα, το  temp_term περιέχει την ακολουθία από string του όρου που θέλουμε τα έγγραφα.
        Στο δεύτερο μέρος του βήματος 4, γίνεται η κατάταξη των αποτελεσμάτων. Χρησιμοποιούνται οι αλγόριθμοι ανάκτησης Boolean retrieval, Vector Space Model (VSM) και Probabilistic retrieval models.
        Το μοντέλο Boolean retrieval υλοποιεί τις Boolean πράξεις, όπως αναφέρθηκαν παραπάνω.
        Το μοντέλο Vector Space Model (VSM) ανακτάει και κατατάζει τα έγγραφα σχετικά με το ερώτημα του χρήστη.  Αρχικά,  το ερώτημα χωρίζεται σε λέξεις, απαλείφονται οι κενοί χαρακτήρες και επεξεργάζονται οι όροι του ερωτήματος με χρήση της συνάρτησης preprocess_text. Χρησιμοποιείται η filter για να αφαιρεθούν οι κενοί χαρακτήρες και όλοι οι όροι αποθηκεύονται στην λίστα terms. Δημιουργεί ένα κενό λεξικό και δημιουργεί ένα αντικείμενο Counter, το οποίο περιέχει τους όρους του ερωτήματος. Στη συνέχεια, υπολογίζεται το tf-idf για κάθε έγγραφο που περιέχει κάποιον από τους όρους του ερωτήματος. Τέλος, γίνεται η ταξινόμηση του κάθε όρου με βάση την τιμή του cosine similarity.
        Το μοντέλο Probabilistic retrieval models ανακτάει και κατατάζει τα έγγραφα σχετικά με το ερώτημα του χρήστη. Αρχικά, φορτώνονται τα δεδομένα του αρχείου processed.json στην μεταβλητή processed. Αν δεν υπάρχει το αρχείο, τότε εκτυπώνεται το μήνυμα ότι δεν μπορεί να υλοποιηθεί η Probabilistic retrieval και τερματίζει η συνάρτηση. Ορίζονται οι παράμετροι του BM25, που είναι ίσοι με k = 1.5 και b = 0.75. Υπολογίζεται ο συνολικός αριθμός των εγγράφων και αποθηκεύεται στην μεταβλητή N. Αρχικοποιείται η μεταβλητή total_words με την τιμή 0, η συγκεκριμένη μεταβλητή υπολογίζει τον συνολικό αριθμό των λέξεων στο έγγραφο. Δημιουργείται ένας βρόχος, ο οποίος ελέγχει αν υπάρχει το πεδίο “Content”. Αν το πεδίο υπάρχει, τότε αποθηκεύεται το κείμενο που αντιστοιχεί στο “Content” στην μεταβλητή content. Ακόμα, διαχωρίζεται το περιεχόμενο του content σε μια λίστα από λέξεις, επιστρέφεται ο αριθμός των λέξεων και αποθηκεύονται στην μεταβλητή total_words. Στην συνέχεια, ελέγχεται αν η μεταβλητή N είναι μεγαλύτερο του μηδενός. Αν ισχύει, τότε υπολογίζεται ο μέσος αριθμός λέξεων, η οποία αντιπροσωπεύεται από την μεταβλητή avgdl. Στην avgdl αποθηκεύεται το αποτέλεσμα του κλάσματος των συνολικών λέξεων δια όλα τον συνολικό αριθμό των εγγράφων. Έπειτα, το ερώτημα χωρίζεται σε λέξεις, απαλείφονται οι κενοί χαρακτήρες και επεξεργάζονται οι όροι του ερωτήματος με χρήση της συνάρτησης preprocess_text. Χρησιμοποιείται η filter για να αφαιρεθούν οι κενοί χαρακτήρες και όλοι οι όροι αποθηκεύονται στην λίστα terms. Δημιουργεί ένα κενό λεξικό για την αποθήκευση των σκορ των εγγράφων. Έπειτα, δημιουργείται ένας βρόχος, ο οποίος διατρέχει κάθε έγγραφο στη λίστα processed. Μέσα στον βρόχο, ελέγχεται αν υπάρχει το πεδίο “Content”. Αν δεν υπάρχει, τότε το έγγραφο παραλείπεται. Εφόσον ανακτάται το κείμενο του πεδίου "Content". Υπολογίζεται ο  αριθμός των λέξεων του εγγράφου. Χρησιμοποιείται η  Counter για να δημιουργήσει ένα λεξικό με τη συχνότητα εμφάνισης κάθε λέξης στο έγγραφο. Αρχικοποιείται η μεταβλητή score με την τιμή 0. Δημιουργείται ένας βρόχος, ο οποίος διατρέχει τον κάθε όρο του εγγράφου. Μέσα στον βρόχο, υπολογίζεται η συχνότητα εμφάνισης του όρου στο έγγραφο με χρήση της doc_term_counter.get(term, 0). Στη συνέχεια, αρχικοποιείται μια μεταβλητή df με την τιμή 0 και μέσα από ένα βρόχο υπολογίζεται σε πόσα έγγραφα εμφανίζεται ο όρος. Αν η μεταβλητή df είναι ίσο με το 0, τότε παραλείπεται. Αλλιώς, υπολογίζεται η idf και η tf από τις μαθηματικές συναρτήσεις:  math.log((N - df + 0.5) / (df + 0.5) + 1) και (ft * (k1 + 1)) / (ft + k1 * (1 - b + b * (doc_length / avgdl))). Επίσης, υπολογίζεται το BM25 score προσθέτοντας το γινόμενο IDF και TF για τον συγκεκριμένο όρο. Τέλος, αποθηκεύεται το υπολογισμένο σκορ στο λεξικό scores με κλειδί το ID του εγγράφου και ταξινομούνται με βάση το score. 
    	Στο βήμα 5 υπολογίζονται τεχνικές για την αξιολόγηση του συστήματος. Αρχικά και στα 3 μοντέλα υπολογίζονται τα έγγραφα, τα οποία είναι συναφή και τα έγγραφα, τα οποία είναι ανακτούνται από την μηχανή αναζήτησης. Στο μοντέλο Boolean retrieval θεωρεί ως συναφή τα έγγραφα στα οποία η αντίστοιχη θέση της λίστας είναι 1. Στο μοντέλο Vector Space Model θεωρεί ως συναφή τα έγγραφα στα οποία το score είναι μεγαλύτερο του 5. Στο μοντέλο Probabilistic retrieval model θεωρεί ως συναφή τα έγγραφα στα οποία το score είναι μεγαλύτερο του 0.1. Υλοποιείται η συνάρτηση evaluate_precision_recall, η οποία υπολογίζει την ακρίβεια, την ανάκληση και το F1-score. Ειδικότερα, δέχεται ως ορίσματα τις λίστες των εγγράφων που ανακτήθηκαν και  των συναφών εγγράφων. Μετριούνται τα έγγραφα που είναι κοινά μεταξύ retrieved_documents και relevant_documents και αποθηκεύονται στην μεταβλητή true_positive. Υπολογίζεται η ακρίβεια ως το πηλικό της μεταβλητής true_positive με τον αριθμό των ανακτηθέντων εγγράφων, εφόσον τα ανακτηθέντα έγγραφα δεν είναι κενά. Αν είναι τότε γίνεται 0. Υπολογίζεται η ανάκληση ως το πηλικό της μεταβλητής true_positive με τον αριθμό των συναφών εγγράφων, εφόσον τα συναφή έγγραφα δεν είναι κενά. Αν είναι τότε γίνεται 0. Υπολογίζεται το F1-score από τον τύπο 2 * ((precision * recall) / (precision + recall)). Στη συνέχεια, υλοποιείται η συνάρτηση calculate_ap, η οποία το AP. Δέχεται ως ορίσματα τις λίστες των εγγράφων που ανακτήθηκαν και  των συναφών εγγράφων. Ελέγχει αν το ανακτηθέν έγγραφο είναι σχετικό και υπολογίζεται η ακρίβεια στο σημείο k. Τέλος, υπολογίζεται ο μέσος όρος ακρίβειας από το αποτέλεσμα του πηλίκου του αθροίσματος της ακρίβειας όλων των εγγράφων δια τον συνολικό αριθμό των συναφών εγγραφών. Τέλος, υλοποιείται η συνάρτηση, η οποία υπολογίζει το mAP. Δέχεται ως ορίσματα τον συνολικό αριθμό των ερωτημάτων και μια λίστα με τις τιμές AP για κάθε ερώτημα. Υπολογίζεται ο μέσος όρος όλων των τιμών AP και επιστρέφεται η τιμή mAP. Ο κώδικας που υλοποιήθηκε:


In [22]:
import json
import math
import string
import pandas as pd
import re
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import customtkinter as ctk

def user_interface():
    print("Καλώς ήρθατε στην μηχανή αναζήτησης για τους ήρωες του 1821!")
    num_query = 0
    ap_score = [] 
    documents =  load_results()
    while True:
        query = input("Εισαγάγετε το ερώτημά σας ή γράψτε exit για έξοδο: ").strip()
        
        if query == 'exit':
            if num_query !=0 :
                mAP = calculate_mAP(num_query,ap_score)
                if mAP != 0.0:
                    print(f"Το mAP score είναι {mAP}")
                else:
                    print("Δεν υπάρχουν αποτελέσματα για υπολογισμό του mAP")
                    
            print("Ευχαριστούμε που χρησιμοποιήσατε τη μηχανη αναζήτησης μας.")
            break

        algorithm_choice = input("Επιλέξτε αλγόριθμο ανάκτησης:\n1) Boolean Retrieval\n2) Vector Space Model (VSM)\n3) Probabilistic Retrieval Models(OKAPI BM25)\nΕπιλογή: ").strip()

        if algorithm_choice == '1':
            # Αυξάνεται ο αριθμός των queries
            num_query = num_query + 1
            retrieved_documents, relevant_documents =boolean_retrieval(query)
            ap_score.append(calculate_ap(retrieved_documents,relevant_documents))
            for document in documents:
                doc_id = document.get('ID')
                if doc_id in relevant_documents:  # Ελέγχει αν το ID είναι στη λίστα relevant_documents
                    title = document.get('First_Title', 'No Title')  # Παίρνει τον τίτλο ή μια προεπιλογή αν δεν υπάρχει
                    content = ""  # Αρχικοποίηση του περιεχομένου
            
                # Δημιουργία του περιεχομένου εκτός του πεδίου "Title" και "ID"
                    for field, field_value in document.items():
                        if field not in ["ID", "First_Title"]:  # Παράβλεψη των πεδίων ID και Title
                            content += f"{field}: {field_value}\n"
            
            # Εισαγωγή του τίτλου και του περιεχομένου στο πλαίσιο
                    print(f"Title: {title}\n\n{content}\n{'-'*50}\n")
            if retrieved_documents == [] and relevant_documents == []:
                print("Δεν υπάρχουν αποτελέσματα!\n\n")
            print(f"Ερώτημα: {query}\nΑποτελέσματα Boolean Retrieval: {retrieved_documents}\nRelevant Documents: {relevant_documents}\n\n")
            
        elif  algorithm_choice == '2':
            # Αυξάνεται ο αριθμός των queries
            num_query = num_query + 1
            ranked_documents,retrieved_documents, relevant_documents = vector_space_model(query)
            ap_score.append(calculate_ap(retrieved_documents,relevant_documents))
            for document in documents:
                doc_id = document.get('ID')
                if doc_id in relevant_documents:  # Ελέγχει αν το ID είναι στη λίστα relevant_documents
                    title = document.get('First_Title', 'No Title')  # Παίρνει τον τίτλο ή μια προεπιλογή αν δεν υπάρχει
                    content = ""  # Αρχικοποίηση του περιεχομένου
            
                # Δημιουργία του περιεχομένου εκτός του πεδίου "Title" και "ID"
                    for field, field_value in document.items():
                        if field not in ["ID", "First_Title"]:  # Παράβλεψη των πεδίων ID και Title
                            content += f"{field}: {field_value}\n"
            
            # Εισαγωγή του τίτλου και του περιεχομένου στο πλαίσιο
                    print(f"Title: {title}\n\n{content}\n{'-'*50}\n")
            if retrieved_documents == [] and relevant_documents == []:
                print("Δεν υπάρχουν αποτελέσματα!\n\n")
            print(f"Ερώτημα: {query}\nΑποτελέσματα Vector Space Model: {ranked_documents}\nRelevant Documents: {relevant_documents}\n\n")
            
        elif algorithm_choice == '3':
            # Αυξάνεται ο αριθμός των queries
            num_query = num_query + 1
            ranked_documents,retrieved_documents, relevant_documents = probabilistic_retrieval(query)
            ap_score.append(calculate_ap(retrieved_documents,relevant_documents))
            for document in documents:
                doc_id = document.get('ID')
                if doc_id in relevant_documents:  # Ελέγχει αν το ID είναι στη λίστα relevant_documents
                    title = document.get('First_Title', 'No Title')  # Παίρνει τον τίτλο ή μια προεπιλογή αν δεν υπάρχει
                    content = ""  # Αρχικοποίηση του περιεχομένου
            
                # Δημιουργία του περιεχομένου εκτός του πεδίου "Title" και "ID"
                    for field, field_value in document.items():
                        if field not in ["ID", "First_Title"]:  # Παράβλεψη των πεδίων ID και Title
                            content += f"{field}: {field_value}\n"
            
            # Εισαγωγή του τίτλου και του περιεχομένου στο πλαίσιο
                    print(f"Title: {title}\n\n{content}\n{'-'*50}\n")
            if retrieved_documents == [] and relevant_documents == []:
                print("Δεν υπάρχουν αποτελέσματα!\n\n")
            print(f"Ερώτημα: {query}\nΑποτελέσματα OKAPI BM25: {ranked_documents}\nRelevant Documents: {relevant_documents}\n\n")
        else:
            print("Μη έγκυρη επιλογή αλγορίθμου. Παρακαλώ εισάγετε έναν έγκυρο αριθμό.")

def boolean_retrieval(query):
    # Φόρτωση του inverted index για το Boolean Retrieval
    inverted_index = load_inverted_index_for_boolean()
    
    # Δημιουργία λιστών για τα έγγραφα που ανακτήθηκαν και τα σχετικά έγγραφα
    retrieved_documents = []
    relevant_documents = []
    
    # Έλεγχος αν το inverted_index είναι κενό ή δεν υπάρχει
    if inverted_index is None:
        print("Η λειτουργία Boolean Retrieval δεν είναι δυνατή λόγω απουσίας αρχείου inverted_index.json.")
        return [], []  # Επιστρέφουμε κενές λίστες αν δεν υπάρχει inverted index
    
    result = []  # Λίστα για να κρατήσουμε τα αποτελέσματα της αναζήτησης
    
    # Έλεγχος για τη λειτουργία "AND" στο ερώτημα
    if "AND" in query:
        query = query.replace('AND', ' ')  # Αντικαθιστούμε το "AND" με κενό
        terms = preprocess_text(query.split())  # Προεπεξεργασία των όρων του ερωτήματος
        terms = list(filter(lambda term: term != '', terms))  # Αφαιρούμε τυχόν κενά από τη λίστα των όρων
                
        result_and = "1" * 20  # Αρχικοποίηση του αποτελέσματος για το "AND"
 
        # Βρόχος για κάθε όρο του ερωτήματος
        for term in terms:
            if term.lower() in inverted_index:  # Αν ο όρος υπάρχει στο inverted index
                result_and = result_and and inverted_index[term]  # Bitwise AND για κάθε όρο
            else:
                return [],[]  # Αν κάποιος όρος δεν υπάρχει, επιστρέφουμε κενές λίστες
        
        print(f"Το αποτέλεσμα από το AND: {result_and}")
        
        # Δημιουργία λίστας των ανακτώμενων και σχετικών εγγράφων
        for i, value in enumerate(result_and):
            if value == '1':  # Αν η τιμή είναι 1, το έγγραφο είναι σχετικό
                relevant_documents.append(f"{i+1}")
                retrieved_documents.append(f"{i+1}")
            else:
               retrieved_documents.append(f"{i+1}")  # Όλα τα έγγραφα προστίθενται στα ανακτημένα
        
        # Μετατροπή σε σύνολα για να αφαιρεθούν οι επαναλήψεις
        relevant_documents = set(relevant_documents)
        retrieved_documents = set(retrieved_documents)
        
        # Υπολογισμός Precision, Recall και F1-score
        precision, recall, F1_score = evaluate_precision_recall(retrieved_documents, relevant_documents)
        
        print(f"Το precision είναι {precision}")
        print(f"Το recall είναι {recall}")
        print(f"Το F1_score είναι {F1_score}")
        
        return retrieved_documents, relevant_documents  # Επιστροφή των αποτελεσμάτων
    
    # Έλεγχος για τη λειτουργία "OR" στο ερώτημα
    elif "OR" in query:
        query = query.replace('OR', ' ')  # Αντικαθιστούμε το "OR" με κενό
        terms = preprocess_text(query.split())  # Προεπεξεργασία των όρων του ερωτήματος
        terms = list(filter(lambda term: term != '', terms))  # Αφαιρούμε τα κενά από τους όρους
        print(f"Εκτέλεση Boolean Retrieval για το ερώτημα: {terms}")
        
        result_or = "0" * 20  # Αρχικοποίηση του αποτελέσματος για το "OR"
     
        # Βρόχος για κάθε όρο του ερωτήματος
        for term in terms:
            if term.lower() in inverted_index:  # Αν ο όρος υπάρχει στο inverted index
                term_result = inverted_index[term.lower()]  # Αποτέλεσμα του όρου
                # Εφαρμογή bitwise OR για κάθε θέση
                result_or = [max(result_or[i], term_result[i]) for i in range(20)]
            else:
                return [],[]  # Αν κάποιος όρος δεν υπάρχει, επιστρέφουμε κενές λίστες
        
        print(f"Αποτέλεσμα από το OR: {result_or}")
        
        # Δημιουργία λίστας των ανακτώμενων και σχετικών εγγράφων
        for i, value in enumerate(result_or):
            if value == '1':  # Αν η τιμή είναι 1, το έγγραφο είναι σχετικό
                relevant_documents.append(f"{i+1}")
                retrieved_documents.append(f"{i+1}")
            else:
               retrieved_documents.append(f"{i+1}")  # Όλα τα έγγραφα προστίθενται στα ανακτημένα
        
        # Μετατροπή σε σύνολα για να αφαιρεθούν οι επαναλήψεις
        relevant_documents = set(relevant_documents)
        retrieved_documents = set(retrieved_documents)
        
        # Υπολογισμός Precision, Recall και F1-score
        precision, recall, F1_score = evaluate_precision_recall(retrieved_documents, relevant_documents)
        
        print(f"Το precision είναι {precision}")
        print(f"Το recall είναι {recall}")
        print(f"Το F1_score είναι {F1_score}")
        
        return retrieved_documents, relevant_documents  # Επιστροφή των αποτελεσμάτων
        
    # Έλεγχος για τη λειτουργία "NOT" στο ερώτημα
    elif "NOT" in query:
        query = query.replace('NOT', ' ')  # Αντικαθιστούμε το "NOT" με κενό
        terms = preprocess_text(query.split())  # Προεπεξεργασία των όρων του ερωτήματος
        terms = list(filter(lambda term: term != '', terms))  # Αφαιρούμε τα κενά από τους όρους
        print(f"Εκτέλεση Boolean Retrieval για το ερώτημα: {terms}")
    
        result_not = "1" * 20  # Ξεκινάμε με όλα τα έγγραφα (όλα "1")

        # Βρόχος για κάθε όρο του ερωτήματος
        for term in terms:
            if term.lower() not in inverted_index:
                # Προσθήκη του όρου στο αποτέλεσμα
                result.append(inverted_index[term.lower()])
            else:
                 return [],[]  # Αν κάποιος όρος δεν υπάρχει, επιστρέφουμε κενές λίστες
            result_not = result_not and inverted_index[term]  # Εφαρμογή bitwise AND για το NOT
        
        print(f"Αποτέλεσμα NOT: {result_not}")
        
        # Αντιστροφή των αποτελεσμάτων του "NOT"
        size_of_list = len(result_not)
        for i in range(size_of_list):
            if result_not[i] == '1':
                result_not[i] = '0'  # Αντιστροφή του "1" σε "0"
            else:
                result_not[i] = '1'  # Αντιστροφή του "0" σε "1"
        
        # Ενημέρωση για τα αποτελέσματα του "NOT"
        temp_term = inverted_index[terms[0]]
        for i in range(size_of_list):
            if temp_term[i] == '0' and result_not[i] == '1':
                result_not[i] = '0'
            
        print(f"Αποτέλεσμα NOT: {result_not}")
        
        # Δημιουργία λίστας των ανακτώμενων και σχετικών εγγράφων
        for i, value in enumerate(result_not):
            if value == '1':  # Αν η τιμή είναι 1, το έγγραφο είναι σχετικό
                relevant_documents.append(f"{i+1}")
                retrieved_documents.append(f"{i+1}")
            else:
               retrieved_documents.append(f"{i+1}")  # Όλα τα έγγραφα προστίθενται στα ανακτημένα
        
        # Μετατροπή σε σύνολα για να αφαιρεθούν οι επαναλήψεις
        relevant_documents = set(relevant_documents)
        retrieved_documents = set(retrieved_documents)
        
        # Υπολογισμός Precision, Recall και F1-score
        precision, recall, F1_score = evaluate_precision_recall(retrieved_documents, relevant_documents)
        
        print(f"Το precision είναι {precision}")
        print(f"Το recall είναι {recall}")
        print(f"Το F1_score είναι {F1_score}")
        
        return retrieved_documents, relevant_documents  # Επιστροφή των αποτελεσμάτων

    else:
        return [],[]  # Αν δεν υπάρχει κανένας έγκυρος τελεστής, επιστρέφουμε κενές λίστες
    
def vector_space_model(query):
    # Φόρτωση του ανεστραμμένου ευρετήριου
    inverted_index = load_inverted_index()

    # Έλεγχος αν το ανεστραμμένο ευρετήριο υπάρχει
    if inverted_index is None:
        print("Η λειτουργία Vector Space Model δεν είναι δυνατή λόγω απουσίας αρχείου inverted_index.json.")
        return

    # Επεξεργασία του ερωτήματος
    terms = preprocess_text(query.split())
    terms = list(filter(lambda term: term != '', terms))  # Αφαίρεση κενών τιμών που δημιουργούνται
    print(f"Εκτέλεση Vector Space Model για το ερώτημα: {terms}")

    # Δημιουργία διανυσμάτων για τα έγγραφα και το ερώτημα
    document_vectors = {}  # Διανύσματα για κάθε έγγραφο
    # Μεταβλητή που περιέχει πόσους όρους έχει το ερώτημα
    query_vector = Counter(terms)
    
    retrieved_documents = []  # Λίστα για τα ανακτημένα έγγραφα
    relevant_documents = []   # Λίστα για τα συναφή έγγραφα

    # Υπολογισμός του διανύσματος για κάθε έγγραφο
    for term, query_term_freq in query_vector.items():
        lower_term = term.lower()  # Μετατροπή του όρου σε πεζά
        if lower_term in inverted_index:
            # Υπολογισμός του IDF (Inverse Document Frequency)
            idf = math.log10(len(inverted_index) / len(inverted_index[lower_term]))

            # Υπολογισμός του tf-idf για κάθε έγγραφο που περιέχει τον όρο
            for doc_id in inverted_index[lower_term]:
                doc_term_freq = inverted_index[lower_term].count(doc_id)  # Συχνότητα όρου στο έγγραφο
                tf_idf = doc_term_freq * idf  # Υπολογισμός tf-idf

                # Ενημέρωση του διανύσματος του εγγράφου
                if doc_id in document_vectors:
                    document_vectors[doc_id] += tf_idf * query_term_freq
                else:
                    document_vectors[doc_id] = tf_idf * query_term_freq

    # Ταξινόμηση των εγγράφων βάση του cosine similarity
    ranked_documents = sorted(document_vectors.items(), key=lambda x: x[1], reverse=True)
    
    # Κατηγοριοποίηση εγγράφων σε σχετικά και μη σχετικά με βάση το αποτελέσμα
    for ID, score in ranked_documents:
        if score >= 5.0:  # Κατώφλι σχετικότητας
            relevant_documents.append(f"{ID}")  # Προσθήκη σε συναφή έγγραφα
            retrieved_documents.append(f"{ID}") # Προσθήκη στα ανακτηθέντα έγγραφα
        else:
            retrieved_documents.append(f"{ID}") # Προσθήκη στα ανακτηθέντα έγγραφα
    
    # Υπολογισμός των μετρικών (precision, recall, F1 score)
    precision, recall, F1_score = evaluate_precision_recall(retrieved_documents, relevant_documents)
    
    # Εμφάνιση των αποτελεσμάτων
    print(f"Το precision είναι {precision}")
    print(f"Το recall είναι {recall}")
    print(f"Το F1_score είναι {F1_score}")        
    
    # Επιστροφή ταξινομημένων εγγράφων και αποτελεσμάτων
    return ranked_documents, retrieved_documents, relevant_documents

def probabilistic_retrieval(query):
    # Φόρτωση των επεξεργασμένων δεδομένων
    processed = load_processed()

    # Έλεγχος αν τα δεδομένα έχουν φορτωθεί σωστά
    if processed is None:
        print("Η λειτουργία Probabilistic Retrieval δεν είναι δυνατή λόγω απουσίας αρχείου processed.json.")
        return

    # Σταθερές για τον υπολογισμό BM25
    k1 = 1.5  # Παράμετρος ρύθμισης για την ενίσχυση της συχνότητας όρων
    b = 0.75  # Παράμετρος ρύθμισης για το μήκος του εγγράφου
    N = len(processed)  # Συνολικός αριθμός εγγράφων
    
    # Υπολογισμός του συνολικού αριθμού λέξεων στο πεδίο "Content"
    total_words = 0
    for doc in processed:
        if "Content" in doc:  # Ελέγχει αν υπάρχει το πεδίο "Content"
            content = doc["Content"]
            total_words += len(content.split())  # Διαχωρισμός σε λέξεις και καταμέτρηση
    
    # Υπολογισμός του avgdl (μέσος αριθμός λέξεων ανά έγγραφο)
    if N > 0:
        avgdl = total_words / N
    else:
        avgdl = 0  # Αν δεν υπάρχουν έγγραφα, το avgdl είναι 0

    # Επεξεργασία του ερωτήματος
    terms = preprocess_text(query.split())
    terms = list(filter(lambda term: term != '', terms))  # Αφαίρεση κενών τιμών
    scores = {}  # Λεξικό για την αποθήκευση των σκορ εγγράφων
    
    # Λίστες για αποθηκευμένα και σχετικά έγγραφα
    retrieved_documents = []
    relevant_documents = []

    # Υπολογισμός BM25 score για κάθε έγγραφο
    for doc in processed:
        if "Content" not in doc:
            continue  # Παράβλεψη εγγράφων χωρίς πεδίο "Content"

        content = doc["Content"]
        doc_length = len(content.split())  # Μήκος εγγράφου
        doc_term_counter = Counter(content.lower().split())  # Καταμέτρηση συχνότητας όρων στο έγγραφο
        score = 0  # Αρχικοποίηση σκορ για το συγκεκριμένο έγγραφο

        for term in terms:
            term = term.lower()  # Μετατροπή του όρου σε πεζά για συνέπεια
            ft = doc_term_counter.get(term, 0)  # Συχνότητα του όρου στο έγγραφο

            # Υπολογισμός Document Frequency (DF)
            df = 0 
            for d in processed:
                if "Content" in d and term in d["Content"].lower():
                    df += 1

            if df == 0:
                continue  # Παράβλεψη όρων που δεν εμφανίζονται σε κανένα έγγραφο

            # Υπολογισμός IDF
            idf = math.log((N - df + 0.5) / (df + 0.5) + 1)

            # Υπολογισμός TF
            tf = (ft * (k1 + 1)) / (ft + k1 * (1 - b + b * (doc_length / avgdl)))

            # Υπολογισμός BM25 score
            score += idf * tf

        # Αποθήκευση του σκορ για το συγκεκριμένο έγγραφο
        scores[doc["ID"]] = score  # Υποθέτουμε ότι τα έγγραφα έχουν πεδίο "ID"
        if score != 0.0:
            retrieved_documents.append(doc["ID"])  # Προσθήκη στο retrieved_documents
        if score >= 0.1:  # Κατώφλι για σχετικά έγγραφα
            relevant_documents.append(doc["ID"])

    # Ταξινόμηση των εγγράφων κατά φθίνουσα σειρά σκορ
    ranked_documents = sorted(scores.items(), key=lambda x: float(x[1]), reverse=True)

    # Εκτύπωση των αποτελεσμάτων
    print(f"Αποτελέσματα Probabilistic Retrieval για το ερώτημα '{query}':")
    for doc_id, score in ranked_documents:
        print(f"Έγγραφο ID: {doc_id}, Σκορ: {score}")
    print(f"Αποτελέσματα από retrieved_documents: {retrieved_documents}")
    print(f"Αποτελέσματα από relevant_documents: {relevant_documents}")
    
    # Υπολογισμός μετρικών (precision, recall, F1 score)
    precision, recall, F1_score = evaluate_precision_recall(retrieved_documents, relevant_documents)
    
    # Εκτύπωση των μετρικών
    print(f"Το precision είναι {precision}")
    print(f"Το recall είναι {recall}")
    print(f"Το F1_score είναι {F1_score}")
    
    # Επιστροφή των ταξινομημένων εγγράφων και των λιστών
    return ranked_documents, retrieved_documents, relevant_documents


def preprocess_text(text):
    # Tokenization: Διαχωρισμός του κειμένου σε λέξεις
    words = word_tokenize(str(text))  # Μετατροπή του κειμένου σε string για αποφυγή προβλημάτων με ακέραιους
    
    processed_words = []  # Λίστα για τις επεξεργασμένες λέξεις
    
    for word in words:
        # Έλεγχος αν η λέξη είναι αριθμός
        if word.isdigit():
            processed_words.append(str(word))  # Αποθήκευση αριθμών ως string
        else:
            # Αφαίρεση σημείων στίξης
            word = word.strip(string.punctuation)
            
            # Stemming: Αφαίρεση καταλήξεων από τις λέξεις
            stemmer = PorterStemmer()
            word = stemmer.stem(word)
            
            # Lemmatization: Εύρεση της ρίζας της λέξης
            lemmatizer = WordNetLemmatizer()
            word = lemmatizer.lemmatize(word)
            
            # Αφαίρεση stop words: Εξάλειψη κοινών λέξεων χωρίς νόημα
            stop_words = set(stopwords.words('english'))
            if word.lower() not in stop_words:
                processed_words.append(word.lower())
    
    return processed_words

def load_inverted_index_for_boolean(file_path='boolean_inverted_index.json'):
    try:
        # Φόρτωση δεδομένων από το αρχείο JSON
        with open(file_path, 'r') as json_file:
            inverted_index = json.load(json_file)
        return inverted_index
    except FileNotFoundError:
        print(f"Το αρχείο {file_path} δεν βρέθηκε.")
        return None
          
def load_processed(file_path='processed.json'):
    try:
        # Φόρτωση δεδομένων από το αρχείο JSON
        with open(file_path, 'r') as processed_file:
            processed = json.load(processed_file)
        return processed
    except FileNotFoundError:
        print(f"Το αρχείο {file_path} δεν βρέθηκε.")
        return None
    
def load_inverted_index(file_path='inverted_index.json'):
    try:
        # Φόρτωση δεδομένων από το αρχείο JSON
        with open(file_path, 'r') as json_file:
            inverted_index = json.load(json_file)
        return inverted_index
    except FileNotFoundError:
        print(f"Το αρχείο {file_path} δεν βρέθηκε.")
        return None

def preprocess_file(text):
    # Tokenization: Διαχωρισμός του κειμένου σε λέξεις
    words = word_tokenize(str(text))  # Μετατροπή του κειμένου σε string
    
    processed_words = []  # Λίστα για τις επεξεργασμένες λέξεις
    
    for word in words:
        # Έλεγχος αν η λέξη είναι αριθμός
        if word.isdigit():
            processed_words.append(str(word))  # Αποθήκευση αριθμών ως string
        else:
            # Αφαίρεση σημείων στίξης
            word = word.strip(string.punctuation)
            if word.lower():  # Έλεγχος αν η λέξη δεν είναι κενή
                processed_words.append(word.lower())
    
    return ' '.join(processed_words)  # Επιστροφή κειμένου ως string
    
def load_results(file_path='results.json'):
    try:
        # Φόρτωση δεδομένων σε DataFrame
        df = pd.read_json(file_path)
        
        # Εφαρμογή προεπεξεργασίας σε κάθε κελί
        for column in df.columns:
            df[column] = df[column].apply(preprocess_file)
        
        # Μετονομασία στηλών
        new_columns = {
            0: 'First_Title',
            1: 'ID',
            2: 'Informations',
            3: 'Date',
            4: 'Content',
        }
        df = df.rename(columns=new_columns)

        # Αφαίρεση μη-ASCII χαρακτήρων από τη στήλη "Content"
        if 'Content' in df.columns:
            df['Content'] = df['Content'].apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', str(x)))

        # Επιστροφή λίστας λεξικών
        return df.to_dict(orient='records')
    except FileNotFoundError:
        print(f"Το αρχείο {file_path} δεν βρέθηκε.")
        return None

def evaluate_precision_recall(retrieved_documents, relevant_documents):
    retrieved_set = set(retrieved_documents)
    relevant_set = set(relevant_documents)
    
    true_positives = len(retrieved_set & relevant_set)  # Κοινά στοιχεία
    precision = true_positives / len(retrieved_set) if retrieved_set else 0
    recall = true_positives / len(relevant_set) if relevant_set else 0
    
    if precision + recall == 0:
        F1_score = 0  # Αποφυγή διαίρεσης με το μηδέν
    else:
        F1_score = 2 * ((precision * recall) / (precision + recall))
    
    return precision, recall, F1_score

# Υπολογίζεται το MAP 

def calculate_mAP(queries,ap_score):
    return sum(ap_score)/queries

#Υπολογίζεται το AP       
def calculate_ap(retrieved, relevant):
    precision_at_k = []  # Λίστα με τις ακρίβειες στα k
    num_relevant = len(relevant)
    num_correct = 0

    for i, item in enumerate(retrieved):
        if item in relevant:  # Έλεγχος αν το στοιχείο είναι σχετικό
            num_correct += 1
            precision_at_k.append(num_correct / (i + 1))  # Υπολογισμός precision@k

    if num_relevant == 0:
        return 0  # Αν δεν υπάρχουν σχετικά στοιχεία
    return sum(precision_at_k) / num_relevant

if __name__ == "__main__":
    #user_interface()
    
    
   # Δημιουργία εφαρμογής
   app = ctk.CTk()
   app.title("Μηχανή Αναζήτησης")

# Ρύθμιση μεγέθους παραθύρου
   app.geometry("700x500")

# Τίτλος στο κέντρο με μεγαλύτερη γραμματοσειρά
   title_label = ctk.CTkLabel(
       app, 
       text="Καλώς ήρθατε στην Μηχανή Αναζήτησης\nγια τους Ήρωες του 1821!", 
       font=("Arial", 20), 
       justify="center"
       )
   title_label.pack(pady=20)

# Πεδίο εισαγωγής ερωτήματος
   query_label = ctk.CTkLabel(app, text="Εισαγάγετε το ερώτημά σας:", font=("Arial", 14))
   query_label.pack(pady=10)
   query_entry = ctk.CTkEntry(app, width=500)
   query_entry.pack(pady=10)

# Επιλογή αλγορίθμου
   algorithm_label = ctk.CTkLabel(app, text="Επιλέξτε αλγόριθμο:", font=("Arial", 14))
   algorithm_label.pack(pady=10)
   algorithm_dropdown = ctk.CTkOptionMenu(app, values=["Boolean Retrieval", "Vector Space Model (VSM)", "Probabilistic Retrieval Models (OKAPI BM25)"])
   algorithm_dropdown.pack(pady=10)

# Περιοχή εμφάνισης αποτελεσμάτων
   results_text = ctk.CTkTextbox(app, width=800, height=400)
   results_text.pack(pady=20)

# Λειτουργικότητα για το κουμπί
   num_query = 0
   ap_scores = []

def on_search():
    global num_query, ap_scores
    query = query_entry.get().strip()
    algorithm_choice = algorithm_dropdown.get()
    documents =  load_results()

    if not query:
        results_text.insert("0.0", "Παρακαλώ εισάγετε ένα ερώτημα.\n")
        return

    num_query += 1
    if algorithm_choice == "Boolean Retrieval":
        retrieved_documents, relevant_documents = boolean_retrieval(query)
        ap_scores.append(calculate_ap(retrieved_documents, relevant_documents))
        
        for document in documents:
            doc_id = document.get('ID')
            if doc_id in relevant_documents:  # Ελέγχει αν το ID είναι στη λίστα relevant_documents
                title = document.get('First_Title', 'No Title')  # Παίρνει τον τίτλο ή μια προεπιλογή αν δεν υπάρχει
                content = ""  # Αρχικοποίηση του περιεχομένου
        
            # Δημιουργία του περιεχομένου εκτός του πεδίου "Title" και "ID"
                for field, field_value in document.items():
                    if field not in ["ID", "First_Title"]:  # Παράβλεψη των πεδίων ID και Title
                        content += f"{field}: {field_value}\n"
        
        # Εισαγωγή του τίτλου και του περιεχομένου στο πλαίσιο
                results_text.insert("0.0", f"Title: {title}\n\n{content}\n{'-'*50}\n")

        if retrieved_documents == [] and relevant_documents == []:
            results_text.insert("0.0","Δεν υπάρχουν αποτελέσματα!\n\n")
        else:
            results_text.insert("0.0", f"Ερώτημα: {query}\nΑποτελέσματα Boolean Retrieval: {retrieved_documents}\nRelevant Documents: {relevant_documents}\n\n")
    elif algorithm_choice == "Vector Space Model (VSM)":
        ranked_documents, retrieved_documents, relevant_documents = vector_space_model(query)
        ap_scores.append(calculate_ap(retrieved_documents, relevant_documents))
        for document in documents:
            doc_id = document.get('ID')
            if doc_id in relevant_documents:  # Ελέγχει αν το ID είναι στη λίστα relevant_documents
                title = document.get('First_Title', 'No Title')  # Παίρνει τον τίτλο ή μια προεπιλογή αν δεν υπάρχει
                content = ""  # Αρχικοποίηση του περιεχομένου
        
            # Δημιουργία του περιεχομένου εκτός του πεδίου "Title" και "ID"
                for field, field_value in document.items():
                    if field not in ["ID", "First_Title"]:  # Παράβλεψη των πεδίων ID και Title
                        content += f"{field}: {field_value}\n"
        
        # Εισαγωγή του τίτλου και του περιεχομένου στο πλαίσιο
                results_text.insert("0.0", f"Title: {title}\n\n{content}\n{'-'*50}\n")
                
                
        if retrieved_documents == [] and relevant_documents == []:
            results_text.insert("0.0","Δεν υπάρχουν αποτελέσματα!\n\n")
        else:        
            results_text.insert("0.0", f"Ερώτημα: {query}\nΑποτελέσματα Vector Space Model: {ranked_documents}\nRelevant Documents: {relevant_documents}\n\n")
    
    elif algorithm_choice == "Probabilistic Retrieval Models (OKAPI BM25)":
        ranked_documents, retrieved_documents, relevant_documents = probabilistic_retrieval(query)
        ap_scores.append(calculate_ap(retrieved_documents, relevant_documents))
        for document in documents:
            doc_id = document.get('ID')
            if doc_id in relevant_documents:  # Ελέγχει αν το ID είναι στη λίστα relevant_documents
                title = document.get('First_Title', 'No Title')  # Παίρνει τον τίτλο ή μια προεπιλογή αν δεν υπάρχει
                content = ""  # Αρχικοποίηση του περιεχομένου
        
            # Δημιουργία του περιεχομένου εκτός του πεδίου "Title" και "ID"
                for field, field_value in document.items():
                    if field not in ["ID", "First_Title"]:  # Παράβλεψη των πεδίων ID και Title
                        content += f"{field}: {field_value}\n"
        
        # Εισαγωγή του τίτλου και του περιεχομένου στο πλαίσιο
                results_text.insert("0.0", f"Title: {title}\n\n{content}\n{'-'*50}\n")
        if retrieved_documents == [] and relevant_documents == []:
            results_text.insert("0.0","Δεν υπάρχουν αποτελέσματα!\n\n")
        else:
            results_text.insert("0.0", f"Ερώτημα: {query}\nΑποτελέσματα OKAPI BM25: {ranked_documents}\nRelevant Documents: {relevant_documents}\n\n")
    else:
        results_text.insert("0.0", "Μη έγκυρη επιλογή αλγορίθμου.\n")
        num_query -= 1

# Κουμπί Αναζήτησης
search_button = ctk.CTkButton(app, text="Αναζήτηση", command=on_search)
search_button.pack(pady=10)

# Κουμπί Εξόδου και Υπολογισμού mAP
def on_exit():
    
    # Ελέγχουμε αν υπάρχουν αποτελέσματα για να υπολογίσουμε το mAP
    if ap_scores:
        mAP = calculate_mAP(num_query, ap_scores)
        results_text.insert("0.0", f"Το mAP score για {num_query} ερωτήματα είναι: {mAP:.2f}\n")
    else:
        results_text.insert("0.0", "Δεν υπάρχουν αποτελέσματα για υπολογισμό του mAP.\n")
    
    results_text.insert("0.0", "Ευχαριστούμε που χρησιμοποιήσατε τη μηχανή αναζήτησης!\n")
    
   # Χρησιμοποιούμε μια καθυστερημένη κλήση για να κλείσουμε το παράθυρο αφού εμφανιστούν τα αποτελέσματα
    app.after(6000, lambda: [app.quit(), app.destroy()])  # Χρησιμοποιούμε lambda για να καλέσουμε τις δύο συναρτήσεις μαζί


exit_button = ctk.CTkButton(app, text="Έξοδος", command=on_exit)
exit_button.pack(pady=10)

# Εκτέλεση της εφαρμογής
app.mainloop()


Το αποτέλεσμα από το AND: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
Το precision είναι 1.0
Το recall είναι 1.0
Το F1_score είναι 1.0


Εισαγάγετε το ερώτημά σας ή γράψτε exit για έξοδο:  greek war
Επιλέξτε αλγόριθμο ανάκτησης:
1) Boolean Retrieval
2) Vector Space Model (VSM)
3) Probabilistic Retrieval Models(OKAPI BM25)
Επιλογή:  2


Εκτέλεση Vector Space Model για το ερώτημα: greek war
Εκτέλεση Vector Space Model για το ερώτημα: ['greek', 'war']
Το precision είναι 1.0
Το recall είναι 1.0
Το F1_score είναι 1.0
Αποτελέσματα Vector Space Model: [('1', 5.6285617984930365), ('2', 5.6285617984930365), ('3', 5.6285617984930365), ('4', 5.6285617984930365), ('5', 5.6285617984930365), ('6', 5.6285617984930365), ('7', 5.6285617984930365), ('8', 5.6285617984930365), ('9', 5.6285617984930365), ('10', 5.6285617984930365), ('11', 5.6285617984930365), ('12', 5.6285617984930365), ('13', 5.6285617984930365), ('14', 5.6285617984930365), ('15', 5.6285617984930365), ('16', 5.6285617984930365), ('17', 5.6285617984930365), ('18', 5.6285617984930365), ('19', 5.6285617984930365), ('20', 5.6285617984930365)]


Εισαγάγετε το ερώτημά σας ή γράψτε exit για έξοδο:  exit


Το mAP score είναι 0.898705676227606
Ευχαριστούμε που χρησιμοποιήσατε τη μηχανη αναζήτησης μας.
